# ✨ Support Ticket Analysis powered by Snowflake Cortex

## 🎯 Purpose
This notebook demonstrates how to analyse customer support tickets using Snowflake's Cortex AI capabilities to identify trends, sentiment patterns, and critical issues that require immediate attention.

## 💡 Why do this?
- 🔍 **Early Issue Detection**: Identify system-wide problems and outages through pattern recognition
- 😊 **Sentiment Analysis**: Track customer satisfaction across multiple dimensions (brand, product, support)
- ⚡ **Automated Escalation**: Use AI to determine which tickets require immediate escalation
- 📈 **Trend Analysis**: Visualize ticket volumes and sentiment patterns over time
- 🚀 **Efficient Support**: Quickly find similar cases using semantic search capabilities

## 🛠️ Solution Components
Note: a combination of SQL & Python will be used for the same outcome. Pick your preferred route!

1. 📊 **Data Visualisation & EDA**
   - Use in-built streamlit functionality to see:
   - Weekly ticket volume by priority
   - Sentiment trends over time

2. 🤖 **AI-Powered Analysis**
   - Issue summarization using `AI_AGG`
   - Automated outage detection with `AI_FILTER`
   - Multi-dimensional sentiment analysis with `AI_SENTIMENT`

3. ⚙️ **Automated Pipeline**
   - CDC for automated ticket processing
   - Scheduled updates for search indices

4. 🎯 **Accurate Retrieval**
   - Semantic search for case similarity matching
   - Quantitative analysis through semantic views 


In [ ]:
# If you're running this on the warehouse runtime, please ensure the following packages are included (see top right)
# snowflake-ml-python
# snowflake
#... and that's it!

import streamlit as st
import pandas as pd
import altair as alt
from datetime import datetime

#notebooks allow for easy context calling - now we have a permissions/role aligned session in flight
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# If you'd like to swap to your data - start by defining the table location here:
table_loc = 'AI_SOL.SUPPORT.RAW_SUPPORT_TICKETS'

## {{}} allows us to reference the table name - even in SQL!

In [ ]:
-- The dummy data we'll be using is Zendesk style data. 
-- We're particularly interested in the TICKET_DESCRIPTION column.
-- NOTE! You can use CTRL+F to quickly replace references to your column.
SELECT * FROM {{table_loc}} LIMIT 10;

In [ ]:
## Let's perform some basic exploration using Streamlit visuals and altair
## NOTE! If you've swapped to your data, remember to select the appropriate columns

# First, let's reference our table
df = session.table(table_loc).to_pandas() 

# Create a form to contain all selection widgets
with st.form("visualization_options"):
    # Get all datetime columns
    date_columns = df.select_dtypes(include=['datetime64']).columns.tolist()
    date_field = st.selectbox('Select Date Field', date_columns)

    # Get categorical columns (object type)
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    category_field = st.selectbox('Select Category Field for Trend Analysis', categorical_columns)

    # Submit button
    submitted = st.form_submit_button("Update Visualizations")

if submitted or True:  # Show default visualization on first load
    # Convert selected date to datetime if needed
    df[date_field] = pd.to_datetime(df[date_field])

    # Display date range and record count
    st.write("Date range in data:", 
             df[date_field].min().strftime('%Y-%m-%d'), 
             "to", 
             df[date_field].max().strftime('%Y-%m-%d'))
    st.write("Number of records:", len(df))

    # Create trend chart by selected category
    trend_by_category = alt.Chart(df).mark_line(opacity=0.6).encode(
        x=alt.X(f'{date_field}:T', 
                title='Week Starting',
                timeUnit='yearweek',  
                axis=alt.Axis(format='%Y-%m-%d')
               ),
        y=alt.Y('count():Q', title='Number of Records'),
        color=alt.Color(f'{category_field}:N', title=category_field),
        tooltip=[
            alt.Tooltip(f'{date_field}:T', title='Week Starting', timeUnit='yearweek', format='%Y-%m-%d'),
            alt.Tooltip(f'{category_field}:N'),
            alt.Tooltip('count():Q', title='Count')
        ]
    ).properties(
        title=f'Weekly Trends by {category_field}',
        height=300
    )

    # Display chart
    st.altair_chart(trend_by_category, use_container_width=True)

    # Show distribution of categories
    distribution = df[category_field].value_counts()
    st.write(f"\nDistribution of {category_field}:")
    st.bar_chart(distribution)

In the dummy dataset, we have considerably more negative events happening around w/c 11th May. Let's use Cortex and AISQL to dig into those values.

We'll start with `AI_AGG` - this allows us to ask a singular prompt across multiple rows of data.

In [ ]:
# Although we loving call it AISQL - 
# you can either use Python or SQL (see next cell!)

from snowflake.snowpark.functions import ai_agg, col, date_trunc

df = session.table(table_loc)

# Filter for the week with the spike in behaviour
# Then, based on those tickets, summarise the findings
result = df.filter(
    date_trunc('week', col('SUBMIT_DATE')) == '2025-05-12'
).agg(
    ai_agg(
        col('TICKET_DESCRIPTION'),
        'What are the top reoccuring issues across these support tickets? Highlight particular trends that would cause negative sentiment issues.'
    )
)

# Display the result
st.write(result.collect()[0][0])


In [ ]:
-- This behaviour acts the same, pick and choose based on your preferred language!
SELECT 
AI_AGG(ticket_description, 
'What are the top reoccuring issues across these support tickets? Highlight particular trends that would cause negative sentiment issues. '
)
FROM {{table_loc}}
WHERE DATE_TRUNC('week',SUBMIT_DATE)='2025-05-12';

In our dummy data, widespread outages appear to be the significant cause for the rise in negative sentiment and overall spike in requests.

Let's use `AI_FILTER` to quickly find customer tickets that mention a system outage as a whole.

Remember, we don't always say what we mean! Using `AI_FILTER` allows us to capture the literal phrase "outage" but also instances such as "everything has gone down" and "nothing works".

We can even use it to apply a degree of business logic - for example "does this issue require escalation?". You may want to provide more detailed guidance in a real life scenario.

In [ ]:
-- We can use AI_FILTER to both narrow down our dataset as well as act as a column boolean.
-- Unsurprisingly in this case, the vast majority of system outage emails results in a recommendation to escalate (with zero guidance to the LLM)

SELECT 
TICKET_ID,
AI_FILTER(PROMPT('Does this ticket require escalation? {0}', TICKET_DESCRIPTION)) as escalate,
TICKET_DESCRIPTION
FROM {{table_loc}}
WHERE
AI_FILTER(PROMPT('Does this ticket mention a system outage? {0}', TICKET_DESCRIPTION))
AND
DATE_TRUNC('week',SUBMIT_DATE)='2025-05-12';

Many products capture a basic sentiment score - but rarely tell the full picture. 

`AI_SENTIMENT` allows for that breadth of view across multiple key factors. As standard, it'll always provide you with the overall view - but equally allow you to drill into the nuance behind that.

For example - a review may be positive overall, but could express concern towards the support team.

In [ ]:
SELECT ticket_id,
AI_SENTIMENT(ticket_description,
    ['brand', 'product', 'customer support']) as sentiment_json,
        sentiment_json:categories[0]:sentiment::STRING AS overall_sentiment,
        sentiment_json:categories[1]:sentiment::STRING as brand_sentiment,
        sentiment_json:categories[2]:sentiment::STRING as product_sentiment,
        sentiment_json:categories[3]:sentiment::STRING as customer_support_sentiment,
        ticket_description
FROM 
{{table_loc}}
WHERE
DATE_TRUNC('week',SUBMIT_DATE)='2025-05-12';

# 🔄 Automated Ticket Processing Pipeline

Now we'll create an automated pipeline using Dynamic Tables to process support tickets and extract key metadata. This pipeline continuously processes new tickets as they arrive, extracting structured information using AI.

This approach replaces the traditional streams and tasks pattern with a more declarative, managed solution that Snowflake automatically orchestrates.


In [ ]:
-- Create automated ticket processing pipeline with Dynamic Table
-- This processes support tickets and extracts key metadata automatically as new tickets arrive
-- Target lag controls how fresh the data is (adjust based on your needs)

CREATE OR REPLACE DYNAMIC TABLE SUPPORT_TICKET_METADATA
  TARGET_LAG = '1 hour'
  WAREHOUSE = tc_wh  -- Update to your warehouse
  AS
WITH ticket_processing AS (
    SELECT 
        t.TICKET_ID as ticket_id,
        t.TICKET_DESCRIPTION as description,
        t.SUBMIT_DATE as submit_date,
        t.PRIORITY as original_priority,
        t.STATUS as status,
        t.CUSTOMER_TIER as customer_tier,
        t.CHANNEL as channel,
        
        -- Extract ticket metadata using AI_COMPLETE with structured output
        AI_COMPLETE(
            model => 'claude-4-sonnet',
            prompt => 'You are an expert in customer support ticket analysis. Analyze this support ticket and extract key information. Only extract information explicitly mentioned in the ticket description.' ||
                      '\n\nSUPPORT TICKET:\n' || 
                      'Ticket ID: ' || t.TICKET_ID || '\n' ||
                      'Description: ' || t.TICKET_DESCRIPTION || '\n' ||
                      'Priority: ' || t.PRIORITY || '\n' ||
                      'Customer Tier: ' || t.CUSTOMER_TIER || '\n' ||
                      '\nEND OF TICKET DATA\n\n',
            response_format => {
                'type': 'json',
                'schema': {
                    'type': 'object',
                    'properties': {
                        'issue_type': {
                            'type': 'string',
                            'description': 'The primary type of issue (e.g., Technical Issue, Billing Question, Feature Request, Bug Report)'
                        },
                        'product_area': {
                            'type': 'string',
                            'description': 'The specific product or service area affected (e.g., Mobile App, Web Platform, API, Billing System)'
                        },
                        'root_cause': {
                            'type': 'string',
                            'description': 'The apparent root cause or source of the issue if mentioned'
                        },
                        'customer_impact': {
                            'type': 'string',
                            'description': 'Description of how the issue impacts the customer (e.g., Cannot access account, Service unavailable, Billing error)'
                        },
                        'requires_escalation': {
                            'type': 'boolean',
                            'description': 'TRUE if the issue requires immediate escalation based on severity or customer impact'
                        },
                        'system_outage': {
                            'type': 'boolean',
                            'description': 'TRUE if the ticket mentions a system-wide outage or widespread service disruption'
                        },
                        'mentions_competitor': {
                            'type': 'boolean',
                            'description': 'TRUE if the customer mentions a competitor or considering switching'
                        },
                        'ticket_summary': {
                            'type': 'string',
                            'description': 'Brief 1-sentence summary of the ticket and customer request'
                        }
                    },
                    'required': ['issue_type', 'product_area', 'customer_impact', 'ticket_summary'],
                    'additionalProperties': false
                }
            }
        ) as metadata_json,
        
        -- Classify urgency level
        AI_CLASSIFY(
            t.TICKET_DESCRIPTION,
            [
                {'label': 'Critical', 'description': 'Service completely unavailable, data loss, security breach, customer threatening to leave'},
                {'label': 'High', 'description': 'Major functionality broken, significant business impact, VIP customer issue'},
                {'label': 'Medium', 'description': 'Feature not working as expected, moderate inconvenience, standard request'},
                {'label': 'Low', 'description': 'Minor issue, cosmetic problem, general question, feature request'}
            ],
            {'task_description': 'Classify support ticket urgency', 'output_mode': 'single'}
        ) as urgency_classification,
        
        -- Classify ticket category
        AI_CLASSIFY(
            t.TICKET_DESCRIPTION,
            [
                {'label': 'Technical Issue', 'description': 'Technical problems, bugs, errors, system not working'},
                {'label': 'Billing', 'description': 'Billing questions, payment issues, invoice problems, refund requests'},
                {'label': 'Feature Request', 'description': 'New feature suggestions, enhancement requests, product improvements'},
                {'label': 'Account Management', 'description': 'Account access, password resets, profile changes, account settings'},
                {'label': 'General Inquiry', 'description': 'General questions, how-to requests, information seeking'}
            ],
            {'task_description': 'Categorize support ticket type', 'output_mode': 'single'}
        ) as category_classification,
        
        -- Analyze multi-dimensional sentiment
        AI_SENTIMENT(
            t.TICKET_DESCRIPTION,
            ['brand', 'product', 'customer support']
        ) as sentiment_json
        
    FROM {{table_loc}} t
)
SELECT 
    ticket_id as TICKET_ID,
    description as ORIGINAL_DESCRIPTION,
    submit_date as SUBMIT_DATE,
    original_priority as ORIGINAL_PRIORITY,
    status as STATUS,
    customer_tier as CUSTOMER_TIER,
    channel as CHANNEL,
    
    -- Extract metadata from JSON response
    metadata_json:issue_type::string as ISSUE_TYPE,
    metadata_json:product_area::string as PRODUCT_AREA,
    metadata_json:root_cause::string as ROOT_CAUSE,
    metadata_json:customer_impact::string as CUSTOMER_IMPACT,
    metadata_json:requires_escalation::boolean as REQUIRES_ESCALATION,
    metadata_json:system_outage::boolean as SYSTEM_OUTAGE,
    metadata_json:mentions_competitor::boolean as MENTIONS_COMPETITOR,
    metadata_json:ticket_summary::string as AI_SUMMARY,
    
    -- Extract classification results
    urgency_classification:labels[0]::string as URGENCY_LEVEL,
    category_classification:labels[0]::string as CATEGORY,
    
    -- Extract sentiment analysis
    TRIM(sentiment_json:categories[0]:sentiment::string) as OVERALL_SENTIMENT,
    TRIM(sentiment_json:categories[1]:sentiment::string) as BRAND_SENTIMENT,
    TRIM(sentiment_json:categories[2]:sentiment::string) as PRODUCT_SENTIMENT,
    TRIM(sentiment_json:categories[3]:sentiment::string) as SUPPORT_SENTIMENT,
    
    -- Derive action flags
    CASE 
        WHEN metadata_json:requires_escalation::boolean = TRUE THEN 'Immediate Escalation Required'
        WHEN metadata_json:system_outage::boolean = TRUE THEN 'System Outage - Priority Response'
        WHEN metadata_json:mentions_competitor::boolean = TRUE THEN 'Retention Risk - Follow Up'
        WHEN TRIM(sentiment_json:categories[0]:sentiment::string) = 'negative' 
             AND urgency_classification:labels[0]::string = 'Critical' THEN 'Critical Negative - Urgent Action'
        ELSE 'Standard Processing'
    END as ACTION_FLAG,
    
    -- Generate next action recommendation
    AI_COMPLETE(
        'claude-4-sonnet',
        'In under 10 words, suggest the next action an agent should take for this support case: ' || description
    ) as NEXT_ACTION
    
FROM ticket_processing;


In [ ]:
-- Query the processed metadata to see the AI-extracted insights
-- This shows how unstructured ticket descriptions are transformed into structured, actionable data

SELECT 
    TICKET_ID,
    ISSUE_TYPE,
    PRODUCT_AREA,
    CUSTOMER_IMPACT,
    URGENCY_LEVEL,
    CATEGORY,
    OVERALL_SENTIMENT,
    ACTION_FLAG,
    NEXT_ACTION,
    AI_SUMMARY
FROM SUPPORT_TICKET_METADATA
ORDER BY SUBMIT_DATE DESC
LIMIT 20;


# AI Powered Case Matching
Cortex Search enables highly accurate search - using base in class retrieval models - to power RAG use cases.

You can create a search service directly in Snowflake, either via the UI (AI > Studio > Cortex Search) - or via SQL similar to the code below.

In [ ]:
CREATE CORTEX SEARCH SERVICE SUPPORT_SEARCH IF NOT EXISTS
  ON ticket_description
  ATTRIBUTES submit_date, ticket_ID
  WAREHOUSE = tc_wh
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        ticket_id,
        submit_date,
        customer_id,
        ticket_description
    FROM {{table_loc}}
);

In [ ]:
-- We can test how the Search Service works either via the UI - or with a simple SQL query

SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'AI_SOL.SUPPORT.SUPPORT_SEARCH',
      '{
        "query": "show me a case where the customer is experiencing internet issues",
        "columns":[
            "ticket_description",
            "submit_date"
        ],
        "limit":1
      }'
  )
)['results'] as results;

In [ ]:
# Now let's test our search service in a simple RAG style scenario
from snowflake.core import Root
from snowflake.cortex import complete

root = Root(session)

search_prompt = "show me a case where the customer is experiencing internet issues"

transcript_search_service = (root
  .databases["ai_sol"]
  .schemas["support"]
  .cortex_search_services["support_search"]
)

resp = transcript_search_service.search(
  query=search_prompt,
  columns=["ticket_id", "ticket_description"],
  limit=1
)


model = 'claude-3-7-sonnet'

llm_call = complete(model,('Give a one line summary and three short key bullet points about this support case. CASE: '+ resp.to_str()))


st.write(llm_call) #LLM response
st.write('---')
st.write(resp.to_json()) #closest matching response

# 📊 AI for Business Intelligence

## 🎯 Purpose
Learn how to leverage Snowflake's AI capabilities for advanced business intelligence through semantic analysis.

## 🔑 Key Components

- **Analyst Tool**: Enables quantitative analysis through natural language queries
- **Data Semantics**: Ensures accuracy and consistency in analysis through defined relationships and metrics
- **Semantic Views**: Can be created through:
  - UI-based configuration in Snowflake interface
  - SQL-based definition (demonstrated in next cell)

## 💡 Benefits
- Natural language querying of your data
- Consistent metric definitions across your organization
- Enhanced data discoverability and understanding
- Improved data governance through semantic layer


If you'd prefer - head to AI > Studio > Cortex Analyst to use the semantic view generator.

![smodel](https://docs.snowflake.com/en/_images/cortex-analyst-semantic-model-overview.png)

In [ ]:
-- The SQL interface makes integration with other platforms far simpler.
-- Now enhanced with AI-extracted metadata for deeper insights
CREATE OR REPLACE SEMANTIC VIEW support_analysis 

TABLES (
  tickets AS AI_SOL.SUPPORT.RAW_SUPPORT_TICKETS 
  PRIMARY KEY (TICKET_ID) 
  WITH SYNONYMS ('support cases', 'customer tickets') 
  COMMENT = 'Main table for support ticket data',
  
  metadata AS AI_SOL.SUPPORT.SUPPORT_TICKET_METADATA
  PRIMARY KEY (TICKET_ID)
  WITH SYNONYMS ('ticket insights', 'ai analysis', 'extracted data')
  COMMENT = 'AI-extracted metadata and insights from support tickets'
)

-- Define the relationship between tickets and metadata
RELATIONSHIPS(
  tickets_to_metadata AS tickets (TICKET_ID) REFERENCES metadata (TICKET_ID)
)

FACTS (
  tickets.response_time AS FIRST_RESPONSE_TIME_HOURS,
  tickets.resolution_time AS RESOLUTION_TIME_HOURS
) 

DIMENSIONS (
  tickets.submit_date AS SUBMIT_DATE COMMENT = 'Date when the ticket was submitted',
  tickets.customer_tier AS CUSTOMER_TIER WITH SYNONYMS ('customer level', 'tier') COMMENT = 'Customer tier level',
  tickets.priority AS PRIORITY COMMENT = 'Ticket priority level. Can be one of HIGH, MEDIUM, or LOW',
  tickets.product_area AS PRODUCT_AREA WITH SYNONYMS ('product category', 'product type') COMMENT = 'Product area related to the ticket',
  tickets.status AS STATUS COMMENT = 'Current status of the ticket',
  tickets.sentiment AS SENTIMENT COMMENT = 'Sentiment analysis of the ticket',
  tickets.classification AS CLASSIFICATION COMMENT = 'Ticket classification category',
  tickets.channel AS CHANNEL COMMENT = 'Channel through which ticket was submitted',
  
  -- AI-extracted metadata dimensions
  metadata.issue_type AS ISSUE_TYPE WITH SYNONYMS ('problem type', 'ticket type') COMMENT = 'AI-classified issue type',
  metadata.product_area AS AI_PRODUCT_AREA WITH SYNONYMS ('affected product', 'service area') COMMENT = 'AI-identified product area affected',
  metadata.root_cause AS ROOT_CAUSE WITH SYNONYMS ('cause', 'source of issue') COMMENT = 'AI-identified root cause',
  metadata.customer_impact AS CUSTOMER_IMPACT WITH SYNONYMS ('impact', 'effect on customer') COMMENT = 'Description of customer impact',
  metadata.urgency_level AS URGENCY_LEVEL WITH SYNONYMS ('urgency', 'criticality') COMMENT = 'AI-classified urgency level',
  metadata.category AS CATEGORY WITH SYNONYMS ('ticket category', 'classification') COMMENT = 'AI-classified ticket category',
  metadata.overall_sentiment AS OVERALL_SENTIMENT WITH SYNONYMS ('sentiment', 'customer mood') COMMENT = 'Overall sentiment analysis',
  metadata.brand_sentiment AS BRAND_SENTIMENT COMMENT = 'Sentiment towards brand',
  metadata.product_sentiment AS PRODUCT_SENTIMENT COMMENT = 'Sentiment towards product',
  metadata.support_sentiment AS SUPPORT_SENTIMENT COMMENT = 'Sentiment towards customer support',
  metadata.action_flag AS ACTION_FLAG WITH SYNONYMS ('priority flag', 'action required') COMMENT = 'Action flag for special handling',
  metadata.requires_escalation AS REQUIRES_ESCALATION COMMENT = 'Whether ticket requires escalation',
  metadata.system_outage AS SYSTEM_OUTAGE WITH SYNONYMS ('outage', 'service disruption') COMMENT = 'Whether ticket mentions system outage',
  metadata.mentions_competitor AS MENTIONS_COMPETITOR WITH SYNONYMS ('competitor mention', 'churn risk') COMMENT = 'Whether customer mentions competitor'
) 

-- Enhanced metrics with AI insights
METRICS (
  tickets.ticket_count AS COUNT(tickets.TICKET_ID) COMMENT = 'Total number of support tickets',
  tickets.avg_response_time AS AVG(tickets.FIRST_RESPONSE_TIME_HOURS) COMMENT = 'Average first response time in hours',
  tickets.avg_resolution_time AS AVG(tickets.RESOLUTION_TIME_HOURS) COMMENT = 'Average resolution time in hours',
  tickets.negative_sentiment_rate AS AVG(
    CASE
      WHEN tickets.SENTIMENT ILIKE '%negative%' THEN 1
      ELSE 0
    END
  ) COMMENT = 'Percentage of tickets with negative sentiment',
  
  -- AI-powered metrics
  metadata.escalation_rate AS AVG(
    CASE
      WHEN metadata.REQUIRES_ESCALATION = TRUE THEN 1
      ELSE 0
    END
  ) COMMENT = 'Percentage of tickets requiring escalation',
  
  metadata.outage_tickets AS COUNT(
    CASE
      WHEN metadata.SYSTEM_OUTAGE = TRUE THEN metadata.TICKET_ID
      ELSE NULL
    END
  ) COMMENT = 'Number of tickets mentioning system outages',
  
  metadata.churn_risk_tickets AS COUNT(
    CASE
      WHEN metadata.MENTIONS_COMPETITOR = TRUE THEN metadata.TICKET_ID
      ELSE NULL
    END
  ) COMMENT = 'Number of tickets with churn risk (competitor mentions)',
  
  metadata.critical_tickets AS COUNT(
    CASE
      WHEN metadata.URGENCY_LEVEL = 'Critical' THEN metadata.TICKET_ID
      ELSE NULL
    END
  ) COMMENT = 'Number of critical urgency tickets',
  
  metadata.negative_brand_sentiment_rate AS AVG(
    CASE
      WHEN metadata.BRAND_SENTIMENT = 'negative' THEN 1
      ELSE 0
    END
  ) COMMENT = 'Percentage of tickets with negative brand sentiment',
  
  metadata.negative_product_sentiment_rate AS AVG(
    CASE
      WHEN metadata.PRODUCT_SENTIMENT = 'negative' THEN 1
      ELSE 0
    END
  ) COMMENT = 'Percentage of tickets with negative product sentiment'
  
) COMMENT = 'Enhanced semantic view for support ticket analysis with AI-extracted insights';

In [ ]:
-- Semantic views can be used by AI features (Analyst & Intelligence)
-- or BI tools via SELECT *.. for example
-- This example shows enhanced analysis with AI-extracted insights

SELECT * FROM SEMANTIC_VIEW(
    support_analysis 
    DIMENSIONS tickets.status, metadata.issue_type, metadata.urgency_level
    METRICS tickets.ticket_count, tickets.avg_response_time, metadata.escalation_rate, metadata.outage_tickets
  );
  
-- Example: Analyze churn risk and sentiment by product area
SELECT * FROM SEMANTIC_VIEW(
    support_analysis
    DIMENSIONS metadata.product_area, metadata.overall_sentiment
    METRICS tickets.ticket_count, metadata.churn_risk_tickets, metadata.negative_brand_sentiment_rate
  )
  WHERE metadata.product_area IS NOT NULL
  ORDER BY metadata.churn_risk_tickets DESC;

# Choose Your Own Adventure

You now have multiple options of where your AI project should live:

1. Continue to test and refine the Semantic View via Analyst Studio. Simply head to AI > Studio > Cortex Analyst, you'll be able to 'speak' to your semantic view, test how it performs based on standard questions, and refine it further.

2. Integrate with a Streamlit application (or similar). Your semantic view is immediately useable by the Cortex Analyst API. You could opt to add SQL generation into an existing application for data discovery.

3. Unleash Agentic Insight. Both the search and semantic services you created can become tools for an AI agent. Let's define one using the Agent UI - you'll be able to use this agent in conjunction with Snowflake Intelligence (PrPr).

![Snowflake Intelligence](https://github.com/sfc-gh-tchristian/AI-Accelerators/blob/main/AI%20Accelerators%20:%20Support%20Ticket%20Analysis/snowintel.png?raw=true)